#Pré-processamento de texto

💻 [acessar este jupyter notebook no colab](https://colab.research.google.com/drive/1v8-HNhYhm0ZIOMGi08_d6m6biUyRKENP?usp=sharing)

O pré-processamento de dados de texto com spaCy e NLTK combina as forças de ambas as bibliotecas para limpar e preparar os dados.

As seguintes tarefas foram realizadas:
* Tokenização
* Remoção de valores *None* dos dados
* Remoção de números e pontuações
* Tratamento de stop words
* Steamming
* Lematização
* Vetorização do texto

##Configurações Iniciais

In [ ]:
#NLTK é nativa no colab, já a biblioteca spaCy é necessário instalação
!pip install -q spacy
!python -m spacy download pt_core_news_sm

In [3]:
#importando as ferramentas necessárias
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

* `punkt` e `stopwords` são usados no NLTK para pré-processar textos, dividindo-os em tokens e removendo palavras comuns, respectivamente.

* `spacy.load('pt_core_news_sm')` carrega um modelo de processamento de linguagem natural para o português que pode ser usado para tarefas avançadas de NLP.

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Tokenização

Tokenização é o processo de dividir um texto em unidades menores chamadas tokens, que podem ser palavras, frases, ou outros elementos significativos. Em outras palavras, é a etapa de "quebrar" o texto em partes que são mais fáceis de analisar e processar.

Por exemplo, dado o texto:
* `"O sol está brilhando no céu."`

Após a tokenização, você pode obter os seguintes tokens:
* `"O", "sol", "está", "brilhando", "no", "céu"`

In [13]:
#base de dados
data = {'text': ['Nós 3 adoramos Python!', 'Python é legal!', 'Ciência de dados é legal', None, 'Estou correndo para aprender Python.']}

df = pd.DataFrame(data)
print(df)

                                   text
0                Nós 3 adoramos Python!
1                       Python é legal!
2              Ciência de dados é legal
3                                  None
4  Estou correndo para aprender Python.


In [14]:
#visualização em forma de tabela
df

,text
0,Nós 3 adoramos Python!
1,Python é legal!
2,Ciência de dados é legal
3,None
4,Estou correndo para aprender Python.


## Remoção de valores *None* dos dados

Remover valores None de um conjunto de dados é uma etapa comum no pré-processamento de dados, especialmente em Python, onde valores `None` podem indicar dados ausentes ou nulos.

In [15]:
df['text'].fillna('', inplace=True)
print (df)

                                   text
0                Nós 3 adoramos Python!
1                       Python é legal!
2              Ciência de dados é legal
3                                      
4  Estou correndo para aprender Python.


In [16]:
df

,text
0,Nós 3 adoramos Python!
1,Python é legal!
2,Ciência de dados é legal
3,
4,Estou correndo para aprender Python.


## Remoção de números e pontuações

A remoção de números e pontuações é uma etapa essencial no pré-processamento de textos, visando limpar e normalizar dados para análise de linguagem natural. Esse processo ajuda a focar no conteúdo textual relevante, eliminando ruídos que podem interferir em análises posteriores.

In [19]:
#limpeza de dados
def remove_noise(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower() #colocando todos os caracteres em minusculo
    return text

df['cleaned_text'] = df['text'].apply(remove_noise)
print(df)

                                   text                         cleaned_text
0                Nós 3 adoramos Python!                 nós  adoramos python
1                       Python é legal!                       python é legal
2              Ciência de dados é legal             ciência de dados é legal
3                                                                           
4  Estou correndo para aprender Python.  estou correndo para aprender python


In [20]:
df

,text,cleaned_text
0,Nós 3 adoramos Python!,nós adoramos python
1,Python é legal!,python é legal
2,Ciência de dados é legal,ciência de dados é legal
3,,
4,Estou correndo para aprender Python.,estou correndo para aprender python


## Tokenizando cada célula

In [22]:
df['tokenized_text'] = df['cleaned_text'].apply(nltk.word_tokenize)
df

,text,cleaned_text,tokenized_text
0,Nós 3 adoramos Python!,nós adoramos python,"[nós, adoramos, python]"
1,Python é legal!,python é legal,"[python, é, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]"
3,,,[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]"


## Tratamento de stop words

O tratamento de stop words é crucial no processamento de linguagem natural, onde palavras comuns e sem significado relevante, como `'o', 'e' e 'a'`, são removidas para melhorar a qualidade das análises textuais. Essa prática ajuda a focar em termos mais significativos, facilitando a extração de informações úteis e a construção de modelos mais precisos.

In [23]:
stop_words = set(stopwords.words('portuguese'))
df['no_stopwords_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word not in stop_words])
df

,text,cleaned_text,tokenized_text,no_stopwords_text
0,Nós 3 adoramos Python!,nós adoramos python,"[nós, adoramos, python]","[adoramos, python]"
1,Python é legal!,python é legal,"[python, é, legal]","[python, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]","[ciência, dados, legal]"
3,,,[],[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]","[correndo, aprender, python]"


In [25]:
#lista de todas as stop words
print(stop_words)

{'for', 'até', 'seremos', 'estivéramos', 'tenho', 'ou', 'estivessem', 'será', 'teus', 'como', 'teríamos', 'também', 'hei', 'com', 'era', 'aqueles', 'sejam', 'houverei', 'tiverem', 'aos', 'sou', 'haja', 'fosse', 'por', 'estivesse', 'tivermos', 'tinha', 'sem', 'ele', 'nossa', 'da', 'vos', 'estejamos', 'meus', 'é', 'nem', 'isso', 'vocês', 'aquele', 'o', 'teu', 'esses', 'seria', 'haver', 'pelos', 'formos', 'tivessem', 'dele', 'das', 'delas', 'nos', 'ela', 'isto', 'ser', 'terei', 'mais', 'tinham', 'tém', 'teve', 'tivera', 'esta', 'às', 'entre', 'fossem', 'houvessem', 'de', 'esse', 'foram', 'houvemos', 'houverão', 'éramos', 'estas', 'as', 'seja', 'houveríamos', 'dos', 'meu', 'tua', 'houvermos', 'me', 'houvéssemos', 'quando', 'tuas', 'nas', 'tenham', 'nossas', 'aquelas', 'houvera', 'que', 'terá', 'estiverem', 'fora', 'uma', 'estivermos', 'essas', 'tenhamos', 'estamos', 'forem', 'minhas', 'somos', 'tínhamos', 'tivéramos', 'estiveram', 'há', 'hajamos', 'estivera', 'e', 'houver', 'pela', 'este',

## Steamming e Lematização

Stemming e lematização são técnicas de normalização de texto usadas em processamento de linguagem natural para reduzir palavras à sua forma base Enquanto o stemming corta sufixos para obter uma raiz comum, podendo resultar em formas truncadas ou não necessariamente válidas, a lematização usa um vocabulário e regras gramaticais para converter palavras em suas formas base ou 'lemmas', preservando o significado e a validade das palavras.

Exemplo:

* O stemmer reduz palavras como "running" e "runners" a uma forma base comum, como "run", que pode ser menos gramaticalmente correta.

* O lemmatizer converte palavras para sua forma base gramaticalmente correta, mantendo "runner" e "running" em formas reconhecíveis e corretas.

In [26]:
stemmer = PorterStemmer()
df['stemmed_text'] = df['no_stopwords_text'].apply(lambda x: [stemmer.stem(word) for word in x])

def lemmatize_text(text):
    doc = nlp(" ".join(text))
    return [token.lemma_ for token in doc]

df['lemmatized_text'] = df['no_stopwords_text'].apply(lemmatize_text)
df

,text,cleaned_text,tokenized_text,no_stopwords_text,stemmed_text,lemmatized_text
0,Nós 3 adoramos Python!,nós adoramos python,"[nós, adoramos, python]","[adoramos, python]","[adoramo, python]","[adorar, python]"
1,Python é legal!,python é legal,"[python, é, legal]","[python, legal]","[python, legal]","[python, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]","[ciência, dados, legal]","[ciência, dado, legal]","[ciência, dado, legal]"
3,,,[],[],[],[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]","[correndo, aprender, python]","[correndo, aprend, python]","[correr, aprender, python]"


## Vetorização do texto (One-Hot Encoding)

One-Hot Encoding é uma técnica de vetorização de texto onde cada palavra é representada como um vetor binário. Cada posição no vetor corresponde a uma palavra no vocabulário, e apenas a posição correspondente à palavra atual é marcada como 1, enquanto as demais são 0. Isso cria uma representação vetorial única e esparsa para cada palavra.

In [27]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['lemmatized_text'])
sequences = tokenizer.texts_to_sequences(df['lemmatized_text'])
one_hot_results = tokenizer.texts_to_matrix(df['lemmatized_text'], mode='binary')
one_hot_results

array([[0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 1.]])

In [28]:
word_index = tokenizer.word_index
print("Word Index:", word_index)

Word Index: {'python': 1, 'legal': 2, 'adorar': 3, 'ciência': 4, 'dado': 5, 'correr': 6, 'aprender': 7}


* Word Embedding (onde cada palavra é um vetor)

In [29]:
max_len = 10
padded_sequences = pad_sequences(sequences, maxlen=max_len)
padded_sequences

array([[0, 0, 0, 0, 0, 0, 0, 0, 3, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 5, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 6, 7, 1]], dtype=int32)

* Contagem de palavras mais comuns

In [30]:
word_freq = Counter([word for sublist in df['lemmatized_text'] for word in sublist])
print("Palavras mais comuns:", word_freq.most_common(5))

Palavras mais comuns: [('python', 3), ('legal', 2), ('adorar', 1), ('ciência', 1), ('dado', 1)]


## Divisão do conjunto de dados para tarefas de Machine Learning

A divisão do conjunto de dados para tarefas de Machine Learning geralmente envolve separar os dados em três partes: treinamento, validação e teste. O conjunto de treinamento é usado para treinar o modelo, o de validação para ajustar hiperparâmetros e o de teste para avaliar o desempenho final do modelo.


In [32]:
X_train, X_temp, y_train, y_temp = train_test_split(df['lemmatized_text'], df['text'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("Tamanho do conjunto de treino:", len(X_train))
print("Tamanho do cojunto de validação:", len(X_val))
print("Tamanho do conjunto de teste:", len(X_test))

Tamanho do conjunto de treino: 3
Tamanho do cojunto de validação: 1
Tamanho do conjunto de teste: 1
